In [1]:
from threading import Thread
import socket
import json
import sys
import pandas as pd
import re
from collections import Counter
import math
import numpy as np
from sklearn.decomposition import PCA
from sklearn.preprocessing import normalize
from sklearn.cluster import KMeans, MeanShift
from matplotlib import pyplot as plt
from numba import jit
from tqdm import tqdm


In [2]:
@jit(forceobj=True,target='cpu',looplift=True)
def similarity(x):
    """ 计算余弦相似度 """
    x=np.array(x)
    x=normalize(x) #单位化
    res=np.dot(x , x.T)
    return res

def calc_prin_eigv(mat, eps):
    """对于给定的矩阵mat，迭代计算其主特征向量"""
    x = np.random.rand(mat.shape[1])
    while True:
        x1 = np.dot(mat, x)
        x1 = x1 / np.linalg.norm(x1)
        if np.abs(x1-x).max() < eps:
            break
        x = x1
    return x



class LocalServer(object):
    def __init__(self, host, port):
        """

        初始化，包括：
        1.读取和预处理数据，构建词典
        2.计算TF-IDF矩阵
        3.计算相似词（或者从已保存好的synonym.txt中读取）

        """
        self.address = (host, port)
        self.data=pd.read_csv('./data/all_news.csv')
        print(len(self.data))

        with open('./data/english', 'r', encoding='utf-8') as f:
            self.stop_words = f.readlines()  # 读取全部内容后，按行存储为list
        self.stop_words = set(i.strip('\n') for i in self.stop_words)

        processed = self.data["body"].apply(self.process) #去除标点，停用词和低频词
        
        self.data['processed']=processed
        self.vocab=self.build_words_Dictionary() #词典
        #print(self.word_dic)

        self.IDF_dict=self.IDF() #词典中的所有词的IDF，构成字典{ word1:IDF1, word2:IDF2, ...}的形式
        vec=self.TF_IDF_vec() #计算文章向量（TF-IDF矩阵），其中的值其实就是TF-IDF值
        self.data['vec']=pd.Series([x for x in vec]) #文章向量

        self.word_similarity() #从已保存的文件中获取词相似度，以做模糊匹配
        #print(self.words_similarity)

        #底下这些代码是将上面的vec取转置，然后再求相似度。
        #vec的每一行代表文章，每一列代表词，因此取转置再求相似度就是求词之间的相似度
        #我只运行了其一次，并将结果存到了synonym.txt中
        #synonym.txt中存的是每个词，与其最相似的前三个词（包括它自己，所以它自己就会是第一个）
        #之后要做模糊匹配，就从synonym.txt中读取即可(当然以下代码还是可运行的)
        '''print("type(vec)==",type(vec))
        tmp=np.array([list(x) for x in vec]).T
        print(tmp.shape)
        tmp=[list(x) for x in tmp]
        #print(tmp)
        
        word_sim=similarity(tmp)
        
        self.word_sim=word_sim
        self.sim_word_of_each_word=[]'''
        
        

        '''wfile=open('./synonym.txt','w')
        
        for i,sim in enumerate(word_sim):
            loc=np.argsort(-sim)[:3]
            now_word=self.vocab[i]
            sim_word=np.array(self.vocab)[loc] #取前三个最相似的
            print(f'{now_word} : {sim_word}',file=wfile)
            self.sim_word_of_each_word.append(sim_word)
        wfile.close()
        self.word_similarity() #从已保存的文件中获取词相似度，以做模糊匹配
        '''
        #本地服务器初始化完成
        print("Local Server has completed initialization!")
        

    def word_similarity(self):
        """直接从synonym.txt中读取相似词，每个词有三个与它最近似的词"""
        rfile=open('./synonym.txt','r')
        content=rfile.readlines()
        rfile.close()
        self.words_similarity={} #原词与其相似词们构成的字典
        for s in content:
            s=s.strip('\n')
            ls=s.split(':')
            word=ls[0].strip(' ') #原词
            ls[1]=ls[1].strip(' ')[1:-1]
            ls[1]=ls[1].split(' ')
            sim_words=[eval(x) for x in ls[1]] #相似词构成的列表
            self.words_similarity[word]=sim_words 
        return 
        
    def process(self,x): 
        """处理正文，去除标点，停用词和低频词"""
        MIN=2 #将出现次数小于MIN的词都视作低频词
        x=re.sub('[^A-Za-z]+', ' ', x).lower()
        x = x.split(' ')
        ls = [z for z in x if z not in self.stop_words]
        cnt=Counter(ls)
        res = [z for z in ls if cnt[z]>=MIN]
        return res
    
    def build_words_Dictionary(self): 
        """构建词典(self.vocab)"""
        vocab=set()
        for pro in self.data['processed']:
            vocab|=set(pro) 
        vocab=sorted(list(vocab))

        wfile=open('./data/vocab.txt','w')
        for word in vocab:
            print(word,file=wfile)
        wfile.close()
        return vocab
    
    def IDF(self):
        """计算词典中每个词的IDF值,构成字典{ word1:IDF1, word2:IDF2, ...}"""
        res={} #初始化空字典
        Y=len(self.data)
        for word in tqdm(self.vocab,desc='IDF'):
            cnt=1
            for processed_data in self.data['processed']:
                if word in processed_data:
                    cnt+=1
            res[word]=math.log(Y/cnt)
        return res

    def TF_IDF_vec(self):
        """计算TF-IDF矩阵"""
        res=np.zeros((len(self.data),len(self.vocab))) #初始化零矩阵
        for i,data in enumerate(tqdm(self.data['processed'],desc='TF-IDF')):
            cnt=Counter(data) #其实就是TF，不过这里的TF还没有除以文章长度，到后面再除
            N=len(data) #文章的长度
            for word in data: #计算每个出现在文章中的词的TF-IDF值
                loc=self.vocab.index(word) #文章向量中，每个词的下标取决于其在词典(self.vocab)中的下标
                tmp=self.IDF_dict[word]*cnt[word]/N
                res[i][loc]=tmp
        return res



    def run(self):
        """
        在服务器端实现合理的并发处理方案，使得服务器端能够处理多个客户端发来的请求
        """
        try:
            server = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
            server.setsockopt(socket.SOL_SOCKET, socket.SO_REUSEADDR, 1)
            server.bind(self.address)
            server.listen(5)
        except socket.error as msg:
            print(msg)
            sys.exit(1)
        print('Waiting connection...')

        while 1:
            conn,addr=server.accept()
            print(f'{conn}connected!')
            t=Thread(target=self.TextSearch,args=(conn,addr))
            t.start()

    def TextSearch(self,conn,addr):
        """
        实现文本检索，以及服务器端与客户端之间的通信
        
        1. 接受客户端传递的数据， 例如检索词
        2. 调用检索函数，根据检索词完成检索
        3. 将检索结果发送给客户端
        
        """

        conn.send(("Connected!").encode())
        text=conn.recv(1024).decode()
        
        words_list=text.split(' ') #检索词列表
        useful_data=np.array(self.data[['title','body','vec','id']])
        res=[] #检索到的内容
        all_title=[] #记录全部标题，从而不会将相同标题的文章两次加到检索内容中
        all_vec=[] #所有检索到的文章的原始文章向量（也就是没PCA降维过的）
        new_words_list=[] #新的检索词列表，其实就是把检索词中，不存在于词典中的去掉，以及将原词的相似词加了进来，
        for word in words_list:
            word=word.lower()
            if word not in self.vocab:
                continue
            new_words_list.extend(self.words_similarity[word]) #原词也总是包含于其相似词列表中，所以不用额外再把原词加到列表中

        for word in new_words_list: #开始检索
            loc=self.vocab.index(word) 
            for data in useful_data:
                tmp=data[2][loc] #该检索词在当前这篇文章中的TF-IDF值
                if data[0] not in all_title and (tmp>0.1 or word in data[0].lower()): 
                    res.append([data[0],data[1],tmp]) #标题，正文，TF-IDF 构成的元组
                    all_title.append(data[0])
                    all_vec.append(data[2])
        
        if all_vec!=[]: #检索到了一些文章

            #计算检索到的文章之间的相似度，再迭代计算主特征向量，
            # 然后将 其 与 每篇文章与其他文章的相似度向量 的余弦相似度作为排序依据（HITS算法）
            useful_similirity=similarity(all_vec) 
            prin_eigv = calc_prin_eigv(useful_similirity, 1e-6) 
            for i,vec in enumerate(useful_similirity):
                res[i][2]=np.dot(prin_eigv,vec) #res[i][2]对应的是TF-IDF值，已经无用了，所以不妨覆盖掉它

            res=sorted(res,key=lambda x:x[2],reverse=True) #降序排序
            res=np.array(res,dtype=object) #将res先变成array，只取res的前两列（标题和正文），然后再返回
            res=res[:,:2]
            res=[tuple(x) for x in res]
            
            conn.send((repr(res)).encode())
            conn.close()

        else: #什么也没检索到，返回空的列表
            conn.send(('[]').encode())
            conn.close()


e:\ANACONDA\lib\site-packages\numba\core\decorators.py:150: NumbaDeprecationWarning: The 'target' keyword argument is deprecated.
  warnings.warn("The 'target' keyword argument is deprecated.", NumbaDeprecationWarning)


In [3]:
server = LocalServer('127.0.0.1', 1234)

2225


TF-IDF: 100%|██████████| 2225/2225 [00:18<00:00, 119.54it/s]


type(vec)== <class 'numpy.ndarray'>
(10131, 2225)
Local Server has completed initialization!


#### 文章聚类与检索结果评测

In [4]:
TF_IDF=normalize(np.array([x for x in server.data['vec']]))
print(TF_IDF,TF_IDF.shape)

km = KMeans(n_clusters=5).fit(TF_IDF)

[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]] (2225, 10131)


In [5]:

# 读取原分类 Y
dict = {'business':0, 'entertainment':1, 'politics':2, 'sport':3, 'tech':4}
Y = np.array([dict[x] for x in server.data['topic']])


def Purity(original, now_label):
    """计算purity"""
    n_ir = np.zeros((len(np.unique(original)), len(np.unique(now_label))))# 属于预定义类i且被分到第r个聚类的文档个数
    
    n_r = np.zeros((len(np.unique(original)),))# 第r个聚类类别的文档个数
    
    for ori, now in zip(original.reshape(now_label.shape), now_label):
        n_ir[ori, now] += 1
        n_r[now] += 1

    P = np.amax(n_ir, axis=0) / n_r
    purity = (n_r / len(now_label) * P).sum()
    return purity


In [6]:
print(Purity(Y,km.labels_))

0.7015730337078652


purity达到了0.7，看上去效果还是不错的。
通过实验，
1.当将MIN设为3时（也就是将文章中出现次数少于3次的词视作低频词），词典的大小大约为6000词，此时purity只有0.5左右
2.将MIN设为2时（也就是现在这种情况），词典大小约为10000词，此时purity有0.7左右
3.所以可以猜测，当MIN设为1时，purity会更高，这意味着通过TF-IDF得到的文章向量是合理的，purity不够高只是因为词典的大小不够大。
（MIN为1的时候，估计要算好一会，所以我没做MIN=1的实验了）

#### 运行服务器端
启动服务器之后，在run.ipynb中运行客户端图形界面

In [7]:
#server = LocalServer('127.0.0.1', 1234)
server.run()

Waiting connection...
<socket.socket fd=4900, family=AddressFamily.AF_INET, type=SocketKind.SOCK_STREAM, proto=0, laddr=('127.0.0.1', 1234), raddr=('127.0.0.1', 57084)>connected!
